In [17]:
import sys
moduleLoc = "/Users/Lykke-AndersenLab/PycharmProjects/AnoThER-Seq"
if moduleLoc in sys.path: 
    print('woo')
else:
    sys.path.append(moduleLoc)
    import matplotlib.pyplot as plt
    
import pandas as pd
import matplotlib, sns

# Enable inline plotting
%matplotlib inline

woo


In [65]:
#read in tail files, filter out decay products by tail length
folderLoc = "/Users/Lykke-AndersenLab/Dropbox/NoctData/"

maxTail = 10
minTail = -100

siLuc = pd.read_csv(folderLoc+"siLuc.tails", index_col=False)
print(siLuc['UniqueReads'].sum())
siLuc = siLuc[siLuc['3Loc']+siLuc['TailLength'] >= minTail]
siLuc = siLuc[siLuc['3Loc']+siLuc['TailLength'] <= maxTail]
print(siLuc['UniqueReads'].sum())

siNoct = pd.read_csv(folderLoc+"siNoct.tails", index_col=False)
print(siNoct['UniqueReads'].sum())
siNoct = siNoct[siNoct['3Loc']+siNoct['TailLength'] >= minTail]
siNoct = siNoct[siNoct['3Loc']+siNoct['TailLength'] <= maxTail]
print(siNoct['UniqueReads'].sum())

NoctWT = pd.read_csv(folderLoc+"NoctWT.tails", index_col=False)
print(NoctWT['UniqueReads'].sum())
NoctWT = NoctWT[NoctWT['3Loc']+NoctWT['TailLength'] >= minTail]
NoctWT = NoctWT[NoctWT['3Loc']+NoctWT['TailLength'] <= maxTail]
print(NoctWT['UniqueReads'].sum())

NoctEA = pd.read_csv(folderLoc+"NoctEA.tails", index_col=False)
print(NoctEA['UniqueReads'].sum())
NoctEA = NoctEA[NoctEA['3Loc']+NoctEA['TailLength'] >= minTail]
NoctEA = NoctEA[NoctEA['3Loc']+NoctEA['TailLength'] <= maxTail]
print(NoctEA['UniqueReads'].sum())

3903782
1456625
4621080
1430617
2411894
812829
2579469
861481


In [56]:
siLuc.head()

,Sequence,UniqueReads,Gene,3Loc,TailLength,TailSequence
0,CAGGGCGAGGCTTATCCATTGCACTCCGGATGTGCTGACCCCTGCG...,84984,RNU1-28P-201|snRNA,0.0,0.0,NaN
1,ATCCGAGGACAATATATTAAATGGATTTTTGGAGCAGGGAGATGGA...,60854,U2.19-201|snRNA,-4.0,1.0,A
2,GTGCTCGCTTCGGCAGCACATATACTAAAATTGGAACGATACAGAG...,46467,RNU6-3P-201|snRNA,0.0,0.0,NaN
3,CTATCCGAGGACAATATATTAAATGGATTTTTGGAGCAGGGAGATG...,23810,U2.19-201|snRNA,-4.0,0.0,NaN
4,CAATGAGGTTTATCCGAGGCGCGATTATTGCTAATTGAAAACTTTT...,16384,RNU4-2-201|snRNA,0.0,3.0,CTG
